# JUNデータcsvの読み込み
member_id	gender	birthday	pref_cd	zipcode	sales_date	part_no	color	size_id	unit_price	list_price	shop_id

In [1]:
# coding: UTF-8
import pandas as pd
import numpy as np
import datetime

df_jun = pd.read_csv('jun_needed_data.csv').iloc[:,1:]

/home/t-matsui/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3,7,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 有効なmember_id
- Nanデータ数 0
- 無効：member_idが0のもの 18153472---(18.2M)
- 有効：member_idが0でないもの 10826978---(10.8M)

In [2]:
print("Nanデータ数", len(df_jun[df_jun['member_id'].isnull()]))
print("member_idが0のもの", len(df_jun.query('member_id == 0')))
print("member_idが0でないもの", len(df_jun.query('member_id != 0')))

Nanデータ数 0
member_idが0のもの 18153472
member_idが0でないもの 10826978


### 有効なshop_id
- Nanデータ数 4660348
- 有効データ数 24320102

In [3]:
a_all = len(df_jun.shop_id)
a = df_jun[df_jun['shop_id'].isnull()]
print("Nanデータ数", len(a))
print("有効データ数", a_all - len(a))

Nanデータ数 4660348
有効データ数 24320102


### 有効なmember_idとshop_idを持つデータのみ抽出
- データ数：6166630---(6.17M)
- 欠損値が一切ないデータの数：1078326--(1.08M)
    - gender == 0: 914
    - gender == 1: 1026732---(1.03M)
    - gender == 2: 50680---(0.05M)

In [4]:
df_jun_valid = df_jun.dropna(subset = ['shop_id']).query('member_id != 0')

In [5]:
print(len(df_jun_valid))
print("欠損値なし", len(df_jun_valid.dropna()))
print("gender == 0", len(df_jun_valid.dropna().query('gender == 0')))
print("gender == 1", len(df_jun_valid.dropna().query('gender == 1')))
print("gender == 2", len(df_jun_valid.dropna().query('gender == 2')))

6166630
欠損値なし 1078326
gender == 0 914
gender == 1 1026732
gender == 2 50680


### 欠損値が完全になく、有効な件数と人数
- 件数：1078326---(1.08M)
- 人数：183953----(0.18M)

In [6]:
df_jun_perfectly_nonna = df_jun_valid.dropna()
print(len(df_jun_perfectly_nonna))
print(len(df_jun_perfectly_nonna.drop_duplicates(subset = 'member_id')))

1078326
183953


### 誕生日を1900年～2020年に。

In [7]:
mask = (df_jun_perfectly_nonna.birthday.astype(np.int) > 19000000) & (df_jun_perfectly_nonna.birthday.astype(np.int) < 20200000)
df_jun_final = df_jun_perfectly_nonna[mask]
df_jun_final['birthday'] = df_jun_final['birthday'].astype(str)
df_jun_final['sales_date'] = df_jun_final['sales_date'].astype(str)

/home/t-matsui/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/t-matsui/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
year = df_jun_final.birthday.str.slice(0,4)
month = df_jun_final.birthday.str.slice(4,6)
day = df_jun_final.birthday.str.slice(6,8)

year = year.astype(int)
month = month.astype(int)
day = day.astype(int)

df_jun_final = df_jun_final[(month > 0) & (month < 13) & (day > 0) & (day < 32)]
print(len(df_jun_final))

1074229


In [9]:
syear = df_jun_final.sales_date.str.slice(0,4)
smonth = df_jun_final.sales_date.str.slice(4,6)
sday = df_jun_final.sales_date.str.slice(6,8)

syear = syear.astype(int)
smonth = smonth.astype(int)
sday = sday.astype(int)

df_jun_final_perfect = df_jun_final[(smonth > 0) & (smonth < 13) & (sday > 0) & (sday < 32)]
print(len(df_jun_final_perfect))

1074229


In [10]:
print(df_jun_final_perfect.sales_date.dtype)
print(df_jun_final_perfect.birthday.dtype)
#print(df_jun_final_perfect.tail(5))

object
object


### indexとsales_dateからなるDataFrameを"today"とする。

In [297]:
today = pd.DataFrame(df_jun_final_perfect['sales_date'].astype(float).astype(np.int))
today.sales_date = pd.to_datetime(today.sales_date, format='%Y%m%d')
print(today.head(10).sales_date.dtype)
print(today.head(10))

datetime64[ns]
   sales_date
44 2017-11-12
45 2017-11-12
46 2017-11-12
47 2017-11-12
57 2015-01-10
58 2015-06-20
59 2016-03-21
60 2016-03-21
61 2016-05-14
62 2016-07-03


In [298]:
birth = pd.DataFrame(df_jun_final_perfect['birthday'].astype(np.int))
birth.birthday = pd.to_datetime(birth.birthday, format='%Y%m%d')
print(birth.head(10).birthday.dtype)
print(birth.head(10))

datetime64[ns]
     birthday
44 1954-07-18
45 1954-07-18
46 1954-07-18
47 1954-07-18
57 1965-04-14
58 1965-04-14
59 1965-04-14
60 1965-04-14
61 1965-04-14
62 1965-04-14


## 年齢を求める関数の定義
入力：dataframe
出力：seriese

In [11]:
def get_Age(x):
    a = datetime.datetime.strftime(x[5], '%Y%m%d')
    b = datetime.datetime.strftime(x[2], '%Y%m%d')

    return int((int(a) - int(b)) / 10000)
    #return x[5] - x[2]

df_jun_final_perfect.head(10).apply(get_Age, axis=1)

TypeError: ("descriptor 'strftime' requires a 'datetime.date' object but received a 'str'", 'occurred at index 44')

In [352]:
df_jun_final_perfect['sales_date'] = today
df_jun_final_perfect['birthday'] = birth
df_jun_final_perfect['age'] = df_jun_final_perfect.apply(get_Age, axis=1)

In [353]:
df_jun_final_perfect.head(10)

,member_id,gender,birthday,pref_cd,zipcode,sales_date,part_no,color,size_id,unit_price,list_price,shop_id,age
44,10000156,2.0,1954-07-18,11.0,3300074.0,2017-11-12,GDC5765,36.0,38,3960.0,4400.0,104284GD,63
45,10000156,2.0,1954-07-18,11.0,3300074.0,2017-11-12,GDK5715,7.0,40,4410.0,4900.0,104284GD,63
46,10000156,2.0,1954-07-18,11.0,3300074.0,2017-11-12,GDK5719,15.0,38,3510.0,3900.0,104284GD,63
47,10000156,2.0,1954-07-18,11.0,3300074.0,2017-11-12,GDM5714,40.0,36,2610.0,2900.0,104284GD,63
57,10000396,1.0,1965-04-14,13.0,1120014.0,2015-01-10,BVC4432,66.0,M,2052.0,3800.0,151092BV,49
58,10000396,1.0,1965-04-14,13.0,1120014.0,2015-06-20,BVS1554,27.0,M,2646.0,4900.0,151092BV,50
59,10000396,1.0,1965-04-14,13.0,1120014.0,2016-03-21,GDM1600,16.0,38,3080.0,4400.0,151090GD,50
60,10000396,1.0,1965-04-14,13.0,1120014.0,2016-03-21,GDM1605,16.0,38,2730.0,3900.0,151090GD,50
61,10000396,1.0,1965-04-14,13.0,1120014.0,2016-05-14,BVH2614,10.0,F,3120.0,3900.0,151092BV,51
62,10000396,1.0,1965-04-14,13.0,1120014.0,2016-07-03,BVC2623,8.0,M,3430.0,4900.0,151092BV,51


## ブランドから性別を割り出す
以下のデータから、
1: 女
2: 男
とわかる。（母数は['member_id', 'gender', 'shop_id']にNaNがないもの）
- GA (female & male)
    - gender 0 347440
    - gender 1 92532
    - gender 2 23151
- GD (female)
    - gender 0 1984381
    - gender 1 454928
    - gender 2 1731
- AN (male)
    - gender 0 133009
    - gender 1 4530
    - gender 2 15691
- GM (male)
    - gender 0 0
    - gender 1 0
    - gender 2 0


In [123]:
df = df_jun.loc[:,['member_id', 'gender', 'shop_id']].dropna()#drop_duplicates(subset = 'member_id')
brand = ('GA', 'GD', 'AN', 'GM')

for b in range(len(brand)):
    print(brand[b])
    df_gender_check = df[df.shop_id.str.contains(brand[b])]
    for i in range(3):
        print("gender", i, len(df_gender_check.query('gender == @i')))

GA
gender 0 347440
gender 1 92532
gender 2 23151
GD
gender 0 1984381
gender 1 454928
gender 2 1731
AN
gender 0 133009
gender 1 4530
gender 2 15691
GM
gender 0 0
gender 1 0
gender 2 0


## genderの特性
- 全データ数:  28980450---(28.98M)
- gender =  0 8977472---(8.977M)
- gender =  1 1665677---(1.666M)
- gender =  2 118515---(0.1185M)
- gender =  3 6
- gender = NaN 18218780---(18.22M)

In [35]:
#df_jun.drop_duplicates(subset = 'gender')
#i = 2
#df_jun.query('gender == @i')
a_all = len(df_jun.iloc[:,1])
print("全データ数: ", a_all)
a = 0
for i in range(4):
    print("gender = ",i, len(df_jun.query('gender == @i')))
    a += len(df_jun.query('gender == @i'))

print("gender = NaN", a_all - a)
#df_jun.query('gender i= NaN')

全データ数:  28980450
gender =  0 8977472
gender =  1 1665677
gender =  2 118515
gender =  3 6
gender = NaN 18218780


### 性別の特性
- 男(gender = 2): 118515---(0.1185M)
- 女(gender = 1): 1665677---(1.666M)
- その他: 27196258---(27.20M)

性別不明があまりにも多い

In [124]:
28980450 - 118515 -  1665677

27196258